In [ ]:
costs_dict = {'Synthetic': [0.03, 0.10, 0.20, 0.30, 0.40, 0.50],
              'Credit': [0.12, 0.23, 0.30, 0.40],
              'Weather': [0.11, 0.19, 0.30, 0.40, 0.48],
              'EMNIST': [0.11, 0.19, 0.25, 0.34, 0.39],
              'CIFAR': [0.08, 0.21, 0.3, 0.38], 
              'IXITiny': [0.08, 0.28, 0.30],
              'ISIC': [0.06, 0.15, 0.19, 0.25, 0.3]
              } 

In [2]:
ROOT_DIR = '/gpfs/commons/groups/gursoy_lab/aelhussein/ot_cost/otcost_fl_rebase'
import pandas as pd
import sys
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bootstrap

In [75]:
def load_results(DATASET):
    with open(f'{ROOT_DIR}/results/{DATASET}_hyperparameter_search.pkl', 'rb') as f :
        results = pickle.load(f)
    return results
    
def bootstrap_ci(data):
    estimates = {}
    for c in data:
        estimates[c]={}
        for arch in data[c]:
            estimates[c][arch]={}
            for optim in data[c][arch]:
                estimates[c][arch][optim]={}
                for lr in data[c][arch][optim]:
                    res = data[c][arch][optim][lr]
                    mean = np.mean(res)
                    bs_reps = bootstrap(np.array(res).reshape(1,-1), statistic=np.mean, n_resamples=1000)
                    ci = bs_reps.confidence_interval[0:2]
                    estimates[c][arch][optim][lr] = np.mean([mean, mean, mean, ci[0], ci[1]])
    return estimates

def best_parameters(results_estimates):
    best_combinations = {}
    for cost, architectures in results_estimates.items():
        for architecture, optimizers in architectures.items():
            best_value = -float('inf')
            best_optimizer_lr = ""
            for optimizer, lrs in optimizers.items():
                for lr, value in lrs.items():
                    if value > best_value:
                        best_value = value
                        best_optimizer_lr = f"{optimizer}: {lr}: {value:.3f}"
            if cost not in best_combinations:
                best_combinations[cost] = {}
            best_combinations[cost][architecture] = best_optimizer_lr
    return best_combinations

def process_results(DATASET):
    results = load_results(DATASET)
    results_estimates = bootstrap_ci(results)
    best_hyperparams = best_parameters(results_estimates)
    return results_estimates, best_hyperparams

### Synthetic

In [86]:
DATASET = 'Synthetic'
results_estimates, best_hyperparams = process_results(DATASET)

In [87]:
best_hyperparams

{0.03: {'single': 'SGD: 0.05: 0.674',
  'joint': 'ADM: 0.1: 0.675',
  'federated': 'ADM: 0.05: 0.675',
  'pfedme': 'ADM: 0.1: 0.654',
  'ditto': 'ADM: 0.1: 0.655'},
 0.1: {'single': 'ADM: 0.1: 0.668',
  'joint': 'ADM: 0.05: 0.665',
  'federated': 'ADM: 0.05: 0.672',
  'pfedme': 'ADM: 0.1: 0.651',
  'ditto': 'ADM: 0.1: 0.657'},
 0.2: {'single': 'ADM: 0.1: 0.619',
  'joint': 'ADM: 0.1: 0.678',
  'federated': 'ADM: 0.1: 0.633',
  'pfedme': 'ADM: 0.05: 0.626',
  'ditto': 'ADM: 0.1: 0.621'},
 0.3: {'single': 'ADM: 0.01: 0.647',
  'joint': 'ADM: 0.05: 0.703',
  'federated': 'ADM: 0.1: 0.705',
  'pfedme': 'ADM: 0.05: 0.664',
  'ditto': 'ADM: 0.1: 0.668'}}

### Credit

In [98]:
DATASET = 'Credit'
results_estimates, best_hyperparams = process_results(DATASET)

### Weather

In [90]:
DATASET = 'Weather'
results_estimates, best_hyperparams = process_results(DATASET)

In [91]:
best_hyperparams

{0.11: {'single': 'SGD: 0.1: 0.895',
  'joint': 'ADM: 0.01: 0.856',
  'federated': 'SGD: 0.1: 0.409',
  'pfedme': 'ADM: 0.05: 0.907',
  'ditto': 'ADM: 0.1: 0.904'},
 0.19: {'single': 'SGD: 0.1: 0.900',
  'joint': 'SGD: 0.1: 0.882',
  'federated': 'SGD: 0.1: 0.288',
  'pfedme': 'ADM: 0.1: 0.925',
  'ditto': 'ADM: 0.1: 0.907'},
 0.3: {'single': 'SGD: 0.1: 0.905',
  'joint': 'ADM: 0.05: 0.209',
  'federated': 'SGD: 0.1: 0.529',
  'pfedme': 'ADM: 0.1: 0.923',
  'ditto': 'ADM: 0.1: 0.911'},
 0.4: {'single': 'SGD: 0.1: 0.898',
  'joint': 'SGD: 0.1: 0.899',
  'federated': 'SGD: 0.1: 0.427',
  'pfedme': 'ADM: 0.1: 0.864',
  'ditto': 'ADM: 0.05: 0.863'}}

### EMNIST

In [92]:
DATASET = 'EMNIST'
results_estimates, best_hyperparams = process_results(DATASET)

In [93]:
best_hyperparams

{0.11: {'single': 'ADM: 0.005: 0.947',
  'joint': 'ADM: 0.01: 0.965',
  'federated': 'ADM: 0.05: 0.603',
  'pfedme': 'ADM: 0.05: 0.957',
  'ditto': 'ADM: 0.05: 0.936'}}

### CIFAR

In [60]:
DATASET = 'CIFAR'
results_estimates, best_hyperparams = process_results(DATASET)

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/commons/groups/gursoy_lab/aelhussein/ot_cost/otcost_fl_rebase/results/CIFAR_hyperparameter_search.pkl'

In [ ]:
best_hyperparams

{0.12: {'single': 'ADM:0.05:0.943',
  'joint': 'ADM:0.1:0.953',
  'federated': 'ADM:0.1:0.934',
  'pfedme': 'ADM:0.1:0.942',
  'ditto': 'ADM:0.1:0.937'}}

### IXITiny

In [ ]:
DATASET = 'IXITiny'
results_estimates, best_hyperparams = process_results(DATASET)

In [ ]:
best_hyperparams

{0.12: {'single': 'ADM:0.05:0.943',
  'joint': 'ADM:0.1:0.953',
  'federated': 'ADM:0.1:0.934',
  'pfedme': 'ADM:0.1:0.942',
  'ditto': 'ADM:0.1:0.937'}}

### ISIC

In [ ]:
DATASET = 'ISIC'
results_estimates, best_hyperparams = process_results(DATASET)

In [ ]:
best_hyperparams

{0.12: {'single': 'ADM:0.05:0.943',
  'joint': 'ADM:0.1:0.953',
  'federated': 'ADM:0.1:0.934',
  'pfedme': 'ADM:0.1:0.942',
  'ditto': 'ADM:0.1:0.937'}}